In [ ]:
import pandas as pd
import toml
import re
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.commands as commands
import paulssonlab.api.benchling as benchling

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Benchling test

In [ ]:
z = reg.sync_benchling(overwrite=True, return_data=True)

In [ ]:
reg.duplicate_collection("LIB", "LT", clear=True)

In [ ]:
reg.registry

# Correctness verification

In [ ]:
x = reg.get("")
x

In [ ]:
with open("/Users/jacob/Downloads/test.gb", "w") as f:
    f.write(x.format("gb"))

# PCR simulation test

In [ ]:
seq1 = "TTTT"
seq1a = "TTTTA"
seq1b = "TTTTC"
seq1c = "CCCCCCCCCCCTTTTC"
seq2 = "AAAATTTTAAAATTTTAAAA"
seq3 = "AGTGATTTTTTTCTCCATTCTTTGTGTGTTTTTTTTGTTTTATGAATTTTTTTAACTGATACCCGTTTTTTTGGAAGGAGACCCGTTTTTTTGGAAG"
seq4 = "TTTTTTAAAAAAAGGGGGGGGGGGGGGTTTTTTCCCCCCCCCCCCAAAAAAATTTTTTAAAAAAA"
seq4p = "GGGGGGGGGGGGGG"

In [ ]:
sequence.find_primer_binding_site(seq4p, seq4)

In [ ]:
sequence.slice_seq?

In [ ]:
sequence.find_primer_binding_site(seq5, p2)

In [ ]:
p1 = "TTTTTAGAAGA"
p2 = "TTTTTTAGAGGG"
seq5 = "AGAAGACCCCGGGGCCCTCT"
sequence.pcr(seq5, p1, p2)

In [ ]:
p1 = "ttttttatggacaaag"
p2 = "ttttttcgatacgggg"
seq6 = "atggacaaagactttgatttcgataatattggcaaacgcaccccgtatcg"
sequence.pcr(seq6, p1, p2)

In [ ]:
sequence.find_primer_binding_site(seq6, p2, try_reverse_complement=True)

In [ ]:
sequence.find_primer_binding_site(
    seq6, sequence.reverse_complement(p2), try_reverse_complement=False
)

In [ ]:
sequence.reverse_complement(seq5)

# GG test

In [ ]:
test_seqs = [reg.get(f"pLIB{plib}")["_seq"] for plib in (1, 82, 23, 95, 110)]

In [ ]:
test_digests = [workflow.re_digest_part(seq, Restriction.BsaI) for seq in test_seqs]

In [ ]:
for seq in test_digests:
    print(seq)

In [ ]:
assembly = sequence.assemble(test_digests)
print(assembly)

In [ ]:
reg.get("oLIB85")

In [ ]:
reg.clear_cache()

In [ ]:
x = reg.get("UNS1_A")
x

In [ ]:
print(x["_seq"])

In [ ]:
print(assembly.circularize())

In [ ]:
with open("/Users/jacob/Downloads/test3.gb", "w") as f:
    f.write(assembly.format("gb"))

# Gibson test

In [ ]:
s1 = "TACTAGAGACCTACTAGTAGCGGCCGCTGCAGGCaaaaaaaaaaaa"
s2 = "aaaaaaaaaaaaGGAGAGAGACCTGCACCATATGCGGTGAGACTAG"

In [ ]:
# gibson two GG'ed TUs
# check we can PCR UNSes onto JUMP vector
# check we can circularize a real 3G rxn

# 3G

# Command parsing

In [ ]:
commands.expr_parser.parse("oLIT1=oLIT2<oLIB1,oLIB2>/BsaI")

In [ ]:
commands.expr_parser.parse(
    "oLIT1=oLIT2<oLIB1,oLIB2>/BsaI,oLIT1=oLIT2<oLIB1,oLIB2>/BsaI"
)

In [ ]:
commands.command_parser.parse(
    "@3(foo, bar, baz, oLIT1=oLIT2<oLIB1,oLIB2>/BsaI, oLIT1=oLIT2<oLIB1,oLIB2>/BsaI)"
)

In [ ]:
command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), pLIB47<oLIB22,oLIB24>/BsaI)"
ast = commands.command_parser.parse(command)
ast

In [ ]:
# command = (
#     "@3G(UNS1-J23101-BCD11-mVenus-L3S3P11-UNS5, UNS5-J23150-CFP-BCD16-L3S2P55-UNS10)"
# )
# command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), pLIB47~oLIB22,oLIB24/BsaI)"
# command = "@Gib(@GG(UNS1, J23101, BCD11, UNS5), @PCR(pLIB47, oLIB22, oLIB24)/BsaI)"
# command = (
#     "@Gib(@GG(UNS1, J23101, BCD11, UNS5), @RE(@PCR(pLIB47, oLIB22, oLIB24), BsaI))"
# )

In [ ]:
# GG
# command = "@GG(J23101, BCD11, mVenus, L3S3P11, p121/BsaI)"
# 3G
# command = "@3G(@GG(UNS1, J23101, BCD11, mVenus, L3S3P11, UNS5), @GG(UNS5, J23150, CFP, BCD16, L3S2P55, UNS10), JUMP_p15a_UNS1_UNS10)"
# command = """@Gib(@GG:tLT(UNS1_A, pTac, B0032m_BC, sigW, L3S1P13, UNS3_E),
#                   @GG:tLT(UNS3_A, J23107_AB, B0034m_BC, mScarlet-I, L3S3P11, UNS4_E),
#                   @GG:tLT(UNS4_A, PsigW, B0032m_BC, sfGFP, L3S2P55, UNS5_E),
#                   @GG:tLT(UNS5_A, pTet_AB, B0034m_BC, rsiW, L3S2P11, UNS6_E),
#                   @GG:tLT(UNS6_A, PsigW, BCD16, sigW, L3S2P21, UNS10_E))"""
# TODO: pLIB213-pLIB218 maps
# TODO: BACKBONE!!!
# command = """@Gib(@GG:tLT3(UNS1_A, pTac, B0032m_BC, sfCFP, L3S1P13, UNS3_E),
#                   @GG:tLT(UNS3_A, J23107_AB, B0034m_BC, mScarlet-I, L3S3P11, UNS4_E))"""
# command = "@GG:tLT(UNS1_A, pTac, B0032m_BC, sfCFP, L3S1P13, UNS3_E)"
command = """@Gib(@GG:tLT(UNS1_A, pTac, B0032m_BC, sfCFP, L3S1P13, UNS3_E),
                  @GG:tLT(UNS3_A, J23107_AB, B0034m_BC, mScarlet-I, L3S3P11, UNS4_E),
                  @GG:tLT(UNS4_A, pTac, B0032m_BC, sfGFP, L3S2P55, UNS5_E),
                  @GG:tLT(UNS5_A, pTet_AB, B0034m_BC, sfCFP, L3S2P11, UNS6_E),
                  @GG:tLT(UNS6_A, pTac, BCD16, sfCFP, L3S2P21, UNS10_E))"""

In [ ]:
commands.command_parser.parse(command)

In [ ]:
# TODO: better error message if next id fails
# ignore columns with formulae when _get_next_empty_row

In [ ]:
workflow.parse_id(" tLIB  ")

In [ ]:
workflow._get_next_empty_row(reg.get_sheet(("tLT", "tus")))

In [ ]:
workflow.get_next_collection_id(reg.get_sheet(("tLT", "tus")))

In [ ]:
def eval_command(cmd, registry, context=None):
    ctx = CommandContext(registry, context)
    return commands.command_parser.parse(cmd, semantics=CommandSemantics(cmds, ctx))


res = eval_command(command, reg, context="pLIB")
print(dissoc(res, "_command"))
print(res["_command"])
unparse_command(res["_command"])